In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [ ]:
# Lee la base de datos
df = pd.read_csv("../data/processed/grd_hmd.csv")
df["fech_alta"] = pd.to_datetime(df["fech_alta"], dayfirst=True)
df = df.sort_values("fech_alta")

# Separa en actividad hospitalaria y Hemodinamia
df_egresos = df.query("tipo_actividad_tipo == 'H'").copy()
df_hmd = df.query("tipo_actividad_tipo == 'A'").copy()

In [ ]:
# Hace recuento de diagnostico
conteo_diagnostico = (
    df_hmd.groupby("ano_egreso")["diag_01_principal_coddes"].value_counts().reset_index()
)
# Hace recuento de procedimientos
conteo_procedimientos = (
    df_hmd.groupby("ano_egreso")["proced_01_principal_coddes"].value_counts().reset_index()
)

In [ ]:
conteo_diagnostico.to_excel("../data/interim/produccion_diagnosticos_hmd.xlsx")

# Obtener trazadoras de Electrofisiología

Una vez se obtuvo la producción de HMD, se observó la producción por diagnóstico en el año 2019 y se clasificó cada uno de ellos como Terapeutico (T), Diagnóstico (D) o de Electrofisiología (E). Esta claificación se guardó en el archivo externo "clasificacion_diags_hmd". Para identificar las trazadoras de Electrofisiología, se filtraron solamente las que tuvieron una clasificación de tal especialidad, y se seleccionaron los 10 diagnósticos más productivos.

In [ ]:
# Lee clasificacion de diagnosticos de HMD, y selecciona el top 10 de Electrofisiologia
clasificacion_diags = pd.read_csv("../data/external/clasificacion_diags_hmd.csv")
trazadoras_electrofisiologia = (
    clasificacion_diags.query("TD == 'E'")
    .sort_values("count", ascending=False)
    .head(10)
)

display(trazadoras_electrofisiologia)

Finalmente, se eliminaron los diagnósticos T82.5 y Z45.0, ya que es muy difícil encontrar su incidencia.

In [ ]:
# Aisla los diagnosticos trazadores de HMD
diagnosticos_electr = list(trazadoras_electrofisiologia["diag_01_principal_coddes"])
diagnosticos_hmd = diagnosticos_electr + [
    "I25.1 - Enfermedad aterosclerótica del corazón",
    "I48.9 - Fibrilacion y flutter auricular, no especificado",
    "I44.7 - Bloqueo de rama izquierda del haz, sin otra especificación",
]

In [ ]:
# Filtra la base de HMD con las trazadoras
trazadoras_hmd = df_hmd.query("diag_01_principal_coddes.isin(@diagnosticos_hmd)")

# Calcula los procedimientos realizados para cada trazadora
procedimientos_de_trazadoras = trazadoras_hmd.groupby(["diag_01_principal_coddes"])[
    "proced_01_principal_coddes"
].value_counts()

In [ ]:
procedimientos_de_trazadoras.to_excel("../data/interim/procedimientos_trazadoras_hmd.xlsx")